In [22]:
import requests
import json
from tqdm import tqdm

import pandas as pd
import os

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
#https://geoscience.data.qld.gov.au/api/action/package_search?q=geochem&rows=1000&start=0
#Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [4]:
# set the API endpoint
api = 'https://geoscience.data.qld.gov.au/api/action/'
# construct our query
query = api + 'package_search?q=reports&rows=1000'
# make the get request and store it in the response object
response = requests.get(query, headers=headers)
# view the payload as JSON
#print(response.text)

if 1 == 1:
    #json_response = response.json()
    json_response = json.loads(response.text)

    #and get a count of results we can retrieve
    total_results = json_response['result']['count']
    print(total_results)
    
    #print(json_response['result']['results'][0])

    #hack without understanding/researching the api too much
    getloop = int(total_results/1000) + 1

    if getloop > 1:
        results_list = []
        for i in range(getloop):
            loopq = query + "&start=" + str(i*1000)
            print(loopq)
            response = requests.get(loopq, headers=headers)
            #json_response = response.json()
            json_response = json.loads(response.text)

            results_list.append(json_response)


93058
https://geoscience.data.qld.gov.au/api/action/package_search?q=reports&rows=1000&start=0
https://geoscience.data.qld.gov.au/api/action/package_search?q=reports&rows=1000&start=1000
https://geoscience.data.qld.gov.au/api/action/package_search?q=reports&rows=1000&start=2000
https://geoscience.data.qld.gov.au/api/action/package_search?q=reports&rows=1000&start=3000
https://geoscience.data.qld.gov.au/api/action/package_search?q=reports&rows=1000&start=4000
https://geoscience.data.qld.gov.au/api/action/package_search?q=reports&rows=1000&start=5000
https://geoscience.data.qld.gov.au/api/action/package_search?q=reports&rows=1000&start=6000
https://geoscience.data.qld.gov.au/api/action/package_search?q=reports&rows=1000&start=7000
https://geoscience.data.qld.gov.au/api/action/package_search?q=reports&rows=1000&start=8000
https://geoscience.data.qld.gov.au/api/action/package_search?q=reports&rows=1000&start=9000
https://geoscience.data.qld.gov.au/api/action/package_search?q=reports&rows=1

https://geoscience.data.qld.gov.au/api/action/package_search?q=reports&rows=1000&start=89000
https://geoscience.data.qld.gov.au/api/action/package_search?q=reports&rows=1000&start=90000
https://geoscience.data.qld.gov.au/api/action/package_search?q=reports&rows=1000&start=91000
https://geoscience.data.qld.gov.au/api/action/package_search?q=reports&rows=1000&start=92000
https://geoscience.data.qld.gov.au/api/action/package_search?q=reports&rows=1000&start=93000


In [5]:
#look at a result for the structure
results_list[0]['result']['results'][887]

{'GeoJSONextent': '{"type":"Polygon","coordinates":[[[138.001184271451,-17.5151699121313],[138.001184437764,-17.5318365462647],[138.001184603078,-17.548503180398],[138.001184813391,-17.5651697445313],[138.017851265247,-17.5651695451638],[138.034517717103,-17.5651693457974],[138.034517515793,-17.5485027356641],[138.05118397365,-17.5485025142989],[138.051183836341,-17.5318358691656],[138.0678503212,-17.5318356318015],[138.067850188891,-17.5151689856681],[138.084516679752,-17.5151687463053],[138.084516555444,-17.4985020991718],[138.101183053306,-17.4985018588102],[138.101182934,-17.4818352096767],[138.117849437863,-17.4818349683162],[138.117849324558,-17.4651683171826],[138.134515834422,-17.4651680738233],[138.134515729118,-17.4485014216897],[138.134515622814,-17.4318347695561],[138.13451551751,-17.4151681174224],[138.151182045378,-17.4151678690642],[138.167848574247,-17.4151676227072],[138.167848479945,-17.3985009665735],[138.151181946075,-17.3985012159306],[138.134515410206,-17.39850146

In [98]:
#each one has organisation - don't really need
#id, , author, spatial, type is useful
#resources - a list of dicts, 
#eg resource:description:
#resource description dictionary
#a resource id, resource:size, resource:, download_url, name, url, resource_type etc

In [99]:
#will try and make a dataframe
#but using the batched results, getting the ids
#turning the resource dictionaries into dataframes, concatenating
#then concatenating back at the end

In [6]:
len(results_list)

94

In [7]:
result_ids = []
for batch in tqdm(results_list, total=len(results_list) ):
    results = batch['result']['results']
    for r in results:
        rid = r['id']
        rauthor = r['author']
        rspatial = r['spatial']
        rtype = r['type']
        
        if len(r['resources']) > 0:
            part_list = []
            for part in r['resources']:
                #modify the columns here to include what you want
                df = pd.DataFrame([part])
                df['rid'] = rid
                df['rauthor'] = rauthor
                df['rspatial'] = rspatial
                df['type'] = rtype
                part_list.append(df)
                allparts = pd.concat(part_list)
                
            result_ids.append(allparts)

dfDatasets = pd.concat(result_ids)            

100%|███████████████████████████████████████████████████████████████████████████████| 94/94 [3:14:34<00:00, 124.19s/it]


In [8]:
dfDatasets.to_csv(r'F:\gid_329265\GSQ_Report_Datasets.csv', index=False)

In [11]:
docx = dfDatasets.loc[dfDatasets['resource:format'] == 'DOCX']
docx.shape

(58367, 38)

In [16]:
docx.size.sum()/1e6

2.217946

In [27]:
def save_link(book_link, book_name):
    the_book = requests.get(book_link, headers=headers, stream=True)
    with open(book_name, 'wb') as f:
      for chunk in the_book.iter_content(1024 * 1024 * 2):  # 2 MB chunks
        f.write(chunk)

#save_link("https://hudoc.echr.coe.int/app/conversion/docx/?library=ECHR&id=001-176931&filename=CASE%20OF%20NDIDI%20v.%20THE%20UNITED%20KINGDOM.docx&logEvent=False","CASE OF NDIDI v. THE UNITED KINGDOM.docx")

In [ ]:
qldir = r'F:\gid_329265\qld' + "\\"
for index, row in tqdm(docx.iterrows(), total=docx.shape[0]):
    url = row['url']
    #print(url)
    urlist = url.split('/')
    name = urlist[-1]
    save_link(url, os.path.join(qldir,name))
    #break

 15%|██████████▌                                                             | 8536/58367 [1:35:20<19:47:50,  1.43s/it]